In [167]:
import warnings
warnings.filterwarnings('ignore')

from tqdm import tqdm
import pandas as pd
import numpy as np
import geopandas as gpd
from geopy.geocoders import Nominatim
import overpass
import requests
import networkx as nx
import osmnx as ox
ox.config(log_console=True, use_cache=True)

from geopy import distance
import networkx as nx

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import LinearRegression

### Задача:

Прогноз продаж одной из популярных моделей [фичерфонов](https://ru.wikipedia.org/wiki/%D0%A4%D0%B8%D1%87%D0%B5%D1%80%D1%84%D0%BE%D0%BD) (на картинке ниже пример похожего устройства) в салонах МегаФона
![](https://39.img.avito.st/640x480/8468720439.jpg)

### Исходные данные:

Датасет содержит следующие поля:

1. `point_id` - Индентификатор салона
2. `lon` - Долгота точки
3. `lat` - Широта точки
4. `target` - Значение таргета, усредненное за несколько месяцев и отнормированное

### Требования к решению и советы:

Ниже приведен список из нескольких важных пунктов, необходимых для решения задания. Выполнение каждого из пунктов влияет на итоговую оценку. Вы можете выполнить каждый из пунктов разными способами, самым лучшим будет считаться вариант, когда всё получение и обработка данных будут реализованы на Питоне (пример: вы можете скачать данные из OSM через интерфейс на сайте overpass-turbo или с помощью библиотек `overpass`/`requests`. Оба варианта будут зачтены, но больше баллов можно заработать во втором случае)



1. Салоны расположены в нескольких разных городах, вам необходимо **определить город для каждого салона** (это понадобится во многих частях задания). К этому есть разные подходы. Вы можете провести [обратное геокодирование](https://en.wikipedia.org/wiki/Reverse_geocoding) с помощью геокодера [Nominatim](https://nominatim.org/), доступного через библиотеку `geopy` примерно вот так:
```python
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="specify_your_app_name_here")
location = geolocator.reverse("52.509669, 13.376294")
print(location.address)
```
В таком случае, вам придется обрабатывать полученную строку адреса, чтобы извлечь название города. Также вы можете скачать из OSM или найти в любом другом источнике границы административно территориальных границ России и пересечь с ними датасет с помощью `geopandas.sjoin` (этот вариант более надежный, но нужно будет разобраться с тем, как устроены границы АТД в OSM, обратите внимание на [этот тег](https://wiki.openstreetmap.org/wiki/Key:admin_level))


2. **Используйте данные OSM**: подумайте, какие объекты могут влиять на продажи фичерфонов. Гипотеза: такие телефоны покупают люди, приезжающие в город или страну ненадолго, чтобы вставить туда отдельную симкарту для роуминга. Можно попробовать использовать местоположения железнодорожных вокзалов (изучите [этот тег](https://wiki.openstreetmap.org/wiki/Tag:railway%3Dstation)). Необходимо использовать хотя бы 5 разных типов объектов из OSM. Скорее всего, вам придется качать данные OSM отдельно для разных городов (см. пример для Нью-Йорка из лекции)


3. **Используйте разные способы генерации признаков**: описать положение салона МегаФона относительно станций метро можно разными способами - найти ***расстояние до ближайшей станции***, или же посчитать, сколько станций попадает в ***500 метровую буферную зону*** вокруг салона. Такие признаки будут нести разную информацию. Так же попробуйте поэкспериментировать с размерами буферных зон (представьте, что значат в реальности радиусы 100, 500, 1000 метров). Попробуйте посчитать расстояние до центра города, до других объектов.

4. **Сделайте визуализации**: постройте 2-3 карты для какого нибудь из городов - как распределен в пространстве таргет, где находятся объекты, полученные вами из OSM. Можете использовать любой инструмент - обычный `plot()`, `folium`, `keplergl`. Если выберете Кеплер, обязательно сохраните в файл конфиг карты, чтобы ее можно было воспроизвести. Сделать это можно вот так:

```python
import json
json_data = kepler_map.config
with open('kepler_config.json', 'w') as outfile:
    json.dump(json_data, outfile)
```
5. Задание не ограничено приведенными выше пунктами, попробуйте нагенерировать интересных признаков, найти в интернете дополнительные данные (в таком случае в комментарии к коду укажите ссылку на ресурс, откуда взяли данные)



6. Это довольно сложная задача - датасет очень маленький, данные по своей природе довольно случайны. Поэтому место и скор на Kaggle не будут играть решающую роль в оценке, но позволят заработать дополнительные баллы

### Read data

In [168]:
train = pd.read_csv('data/mf_geo_train.csv')
test = pd.read_csv('data/mf_geo_test.csv')

In [169]:
train.head(2)

,point_id,lon,lat,target
0,ommNZCUV,37.590776,55.848630,-0.348157
1,nMe2LHPb,37.784210,55.750271,1.294206


In [170]:
test.head(2)

,point_id,lon,lat,target
0,F4lXR1cG,37.681242,55.74804,0.0091
1,4LJu4GTf,60.580910,56.79586,0.0091


In [171]:
df = pd.concat([train, test], axis=0)

# Определяем город для каждого салона

In [172]:
geolocator = Nominatim(user_agent="myapp")

In [173]:
coordinates = np.stack((df.lat.values, df.lon.values), axis=-1)

In [174]:
cities = []
for i in range(coordinates.shape[0]):
    location = geolocator.reverse(coordinates[i])
    try:
        cities.append(location.raw['address']['city'])
    except:
        cities.append(location.raw['address']['state'])

In [175]:
df['city']=cities

In [176]:
df['city'].value_counts()

Москва                         196
Санкт-Петербург                 99
Самара                          34
Казань                          32
Екатеринбург                    27
Ростов-на-Дону                  26
Нижний Новгород                 26
Красноярск                      25
Уфа                             24
городской округ Новосибирск     17
Новосибирск                     16
Колпино                          4
Зеленоград                       4
Московская область               1
Пушкин                           1
Name: city, dtype: int64

In [177]:
df['city'][df['city'] == 'Московская область'] = 'Москва'
df['city'][df['city'] == 'Пушкин'] = 'Санкт-Петербург'
df['city'][df['city'] == 'Зеленоград'] = 'Москва'
df['city'][df['city'] == 'Колпино'] = 'Санкт-Петербург'
df['city'][df['city'] == 'городской округ Новосибирск'] = 'Новосибирск'

In [178]:
df['city'].value_counts()

Москва             201
Санкт-Петербург    104
Самара              34
Новосибирск         33
Казань              32
Екатеринбург        27
Ростов-на-Дону      26
Нижний Новгород     26
Красноярск          25
Уфа                 24
Name: city, dtype: int64

# Downtown distance

In [179]:
cities = df['city'].value_counts().keys()

In [180]:
downtowns = dict.fromkeys(cities)

In [181]:
overpass_url = "http://overpass-api.de/api/interpreter"

for city in cities:
    overpass_query = '''
    [out:json][timeout:60];
    (
      node["place"="city"]["name"= "{}"];
    );
    out body;
    '''.format(city)
    response = requests.get(overpass_url, 
                            params={'data': overpass_query})
    downtown = response.json()

    downtown_osm = pd.DataFrame(downtown['elements'])
    downtown_osm = downtown_osm.join(
        pd.DataFrame([x['tags'] for x in downtown['elements']]), lsuffix='_left', rsuffix='_right')
    downtowns[city] = np.stack((float(downtown_osm.lat),
                                float(downtown_osm.lon)), axis=-1)

In [182]:
downtowns

{'Москва': array([55.7504461, 37.6174943]),
 'Санкт-Петербург': array([59.938732, 30.316229]),
 'Самара': array([53.198627, 50.113987]),
 'Новосибирск': array([55.0282171, 82.9234509]),
 'Казань': array([55.7823547, 49.1242266]),
 'Екатеринбург': array([56.839104, 60.60825 ]),
 'Ростов-на-Дону': array([47.2213858, 39.7114196]),
 'Нижний Новгород': array([56.328571, 44.003506]),
 'Красноярск': array([56.0090968, 92.8725147]),
 'Уфа': array([54.7261409, 55.947499 ])}

In [183]:
downtown_dist = np.zeros(coordinates.shape[0])

for i in tqdm(range(coordinates.shape[0])):
    city = str(df.iloc[i]['city'])
    downtown_dist[i] = distance.great_circle(coordinates[i],
                                             downtowns[city]).miles

df['downtown_dist'] = downtown_dist

100%|██████████| 532/532 [00:00<00:00, 9991.63it/s]


In [185]:
def city_bbox(city):
    city_df = df[df['city']==city]
    lat_coating = city_df.lat.max() - city_df.lat.min()
    lon_coating = city_df.lon.max() - city_df.lon.min()
     
    lat_min = downtowns[city][0] - lat_coating
    lon_min = downtowns[city][1] - lon_coating
    lat_max = downtowns[city][0] + lat_coating
    lon_max = downtowns[city][1] + lon_coating
    return lat_min,lon_min,lat_max,lon_max

# Railway stations

In [231]:
stations_data = []
for city in tqdm(cities):
    bbox = city_bbox(city)
    overpass_url = "https://overpass.openstreetmap.ru/api/interpreter"

    overpass_query = '''
    [out:json][timeout:60];
    (
      node["railway"="station"]["train"="yes"]["network"!="МЦК"]({},{},{},{});
    );
    out body;
    '''.format(*bbox)
    response = requests.get(overpass_url, 
                            params={'data': overpass_query})
    railways = response.json()

    railways_osm = pd.DataFrame(railways['elements'])
    railways_osm = railways_osm.join(
        pd.DataFrame([x['tags'] for x in railways['elements']]), lsuffix='_left', rsuffix='_right')
    
    try:
        railways_osm = railways_osm[['lat', 'lon', 'name']]
        railways_osm['city'] = city
        stations_data.append(railways_osm)
    except:
        continue

100%|██████████| 10/10 [00:09<00:00,  1.05it/s]


In [232]:
railways_osm = pd.concat(stations_data, axis=0)

In [233]:
railways_osm

,lat,lon,name,city
0,55.757774,37.662184,Москва-Пассажирская-Курская,Москва
1,55.774248,37.659914,Москва-Пассажирская-Казанская,Москва
2,55.415149,37.896829,Аэропорт-Домодедово,Москва
3,55.729464,37.212924,Усово,Москва
4,55.780127,37.862776,Стройка,Москва
...,...,...,...,...
2,56.264623,43.757612,Доскино,Нижний Новгород
3,56.322165,43.945693,Нижний Новгород-Московский,Нижний Новгород
0,55.998447,92.951680,Злобино,Красноярск
1,55.980004,92.841601,Енисей,Красноярск


In [189]:
rs_coords = np.stack((railways_osm.lat.values, railways_osm.lon.values), axis=-1)

In [190]:
rail_station = np.zeros(coordinates.shape[0])

for i in tqdm(range(coordinates.shape[0])):
    for j in range(rs_coords.shape[0]):
        if distance.great_circle(coordinates[i],
                                 rs_coords[j]).miles <= 1.5:
            rail_station[i] = 1
            break
            
df['rail_station'] = rail_station
df['rail_station'].value_counts()

100%|██████████| 532/532 [00:00<00:00, 1226.56it/s]


0.0    383
1.0    149
Name: rail_station, dtype: int64

# Airports

In [234]:
airports_data = []
for city in tqdm(cities):
    bbox = city_bbox(city)    
    overpass_url = "https://overpass.openstreetmap.ru/api/interpreter"

    overpass_query = '''
    [out:json][timeout:60];
    (
      node["aeroway"="aerodrome"]({},{},{},{});
    );
    out body;
    '''.format(*bbox)
    response = requests.get(overpass_url, 
                            params={'data': overpass_query})
    airports = response.json()

    airports_osm = pd.DataFrame(airports['elements'])
    airports_osm = airports_osm.join(
        pd.DataFrame([x['tags'] for x in airports['elements']]), lsuffix='_left', rsuffix='_right')
    
    try:
        airports_osm = airports_osm[['lat', 'lon', 'name']]
        airports_osm['city'] = city
        airports_data.append(airports_osm)
    except:
        continue

100%|██████████| 10/10 [00:08<00:00,  1.17it/s]


In [235]:
airports_osm = pd.concat(airports_data, axis=0)

In [236]:
airports_osm

,lat,lon,name,city
0,55.837056,38.170611,Аэродром Монино,Москва
1,55.421701,38.229232,Аэродром Бронницы,Москва
2,56.235440,37.554430,Батюшково,Москва
0,60.203141,30.333713,Касимово,Санкт-Петербург
1,60.014495,29.704800,Бычье поле,Санкт-Петербург
2,59.872040,30.803225,Манушкино,Санкт-Петербург
3,60.033364,30.015391,Горская,Санкт-Петербург
4,59.617509,29.542611,Сельцо,Санкт-Петербург
0,53.224161,50.329398,Безымянка (аэродром),Самара
1,53.250000,50.053000,Аэродром Рождествено,Самара


In [194]:
air_coords = np.stack((airports_osm.lat.values, airports_osm.lon.values), axis=-1)

In [195]:
airport = np.zeros(coordinates.shape[0])

for i in tqdm(range(coordinates.shape[0])):
    for j in range(air_coords.shape[0]):
        if distance.great_circle(coordinates[i],
                                 air_coords[j]).miles <= 5:
            airport[i] = 1
            break
            
df['airport'] = airport
df['airport'].value_counts()

100%|██████████| 532/532 [00:00<00:00, 5737.32it/s]


0.0    506
1.0     26
Name: airport, dtype: int64

# Hotels

In [237]:
hotels_data = []
for city in tqdm(cities):
    bbox = city_bbox(city)
    overpass_url = "https://overpass.openstreetmap.ru/api/interpreter"

    overpass_query = '''
    [out:json][timeout:60];
    (
      node["tourism"="hotel"]({},{},{},{});
    );
    out body;
    '''.format(*bbox)
    response = requests.get(overpass_url, 
                            params={'data': overpass_query})
    hotels = response.json()

    hotels_osm = pd.DataFrame(hotels['elements'])
    hotels_osm = hotels_osm.join(
        pd.DataFrame([x['tags'] for x in hotels['elements']]), lsuffix='_left', rsuffix='_right')
    
    hotels_osm = hotels_osm[['lat', 'lon', 'name']]
    hotels_osm['city'] = city
    hotels_data.append(hotels_osm)

100%|██████████| 10/10 [00:08<00:00,  1.18it/s]


In [238]:
hotels_osm = pd.concat(hotels_data, axis=0)

In [239]:
hotels_osm

,lat,lon,name,city
0,55.636191,37.995574,"Бронхолёгочный санаторий ""Ранее детство""",Москва
1,55.780626,37.619490,Славянка,Москва
2,55.592740,38.121216,Романтика,Москва
3,55.598946,38.118072,Европа,Москва
4,55.733201,37.644690,Swissotel Красные холмы,Москва
...,...,...,...,...
15,54.761476,56.049214,NaN,Уфа
16,54.822362,56.074753,Караидель,Уфа
17,54.822180,56.082810,Альфа,Уфа
18,54.736766,55.971217,Доплайн,Уфа


In [201]:
hotels_coords = np.stack((hotels_osm.lat.values, hotels_osm.lon.values), axis=-1)

In [276]:
hotels = np.zeros(coordinates.shape[0])

for i in tqdm(range(coordinates.shape[0])):
    for j in range(hotels_coords.shape[0]):
        if distance.great_circle(coordinates[i],
                                 hotels_coords[j]).miles <= 0.5:
            hotels[i] += 1
            
df['hotels'] = hotels
pd.DataFrame(df['hotels'].value_counts())

100%|██████████| 532/532 [00:07<00:00, 72.21it/s]


,hotels
0.0,257
1.0,123
2.0,46
3.0,37
4.0,12
5.0,11
7.0,8
13.0,4
18.0,3
8.0,3


# Other Phone Shops

In [240]:
shops_data = []
for city in tqdm(cities):
    bbox = city_bbox(city)
    overpass_url = "https://overpass.openstreetmap.ru/api/interpreter"

    overpass_query = '''
    [out:json][timeout:60];
    (
      node["brand"!="Мегафон"]["shop"="mobile_phone"]({},{},{},{});
    );
    out body;
    '''.format(*bbox)
    response = requests.get(overpass_url, 
                            params={'data': overpass_query})
    phone_shops = response.json()

    phone_shops_osm = pd.DataFrame(phone_shops['elements'])
    phone_shops_osm = phone_shops_osm.join(
        pd.DataFrame([x['tags'] for x in phone_shops['elements']]), lsuffix='_left', rsuffix='_right')
    
    phone_shops_osm = phone_shops_osm[['lat', 'lon', 'name']]
    phone_shops_osm['city'] = city
    shops_data.append(phone_shops_osm)

100%|██████████| 10/10 [00:07<00:00,  1.31it/s]


In [241]:
phone_shops_osm = pd.concat(shops_data, axis=0)

In [242]:
phone_shops_osm

,lat,lon,name,city
0,55.870407,37.662253,МТС,Москва
1,55.877419,38.104891,Сотовая связь,Москва
2,55.869770,37.665891,Связной,Москва
3,55.870280,37.663685,Связной,Москва
4,55.916902,37.997222,МегаФон,Москва
...,...,...,...,...
36,54.748945,56.021831,phone24,Уфа
37,54.756794,56.035819,M_mobile,Уфа
38,54.733733,55.966335,Kingstore,Уфа
39,54.742251,55.957406,Бум Рум,Уфа


In [206]:
ps_coords = np.stack((phone_shops_osm.lat.values, phone_shops_osm.lon.values), axis=-1)

In [207]:
phone_shops = np.zeros(coordinates.shape[0])

for i in tqdm(range(coordinates.shape[0])):
    for j in range(ps_coords.shape[0]):
        if distance.great_circle(coordinates[i],
                                 ps_coords[j]).miles <= 0.5:
            phone_shops[i] += 1
            
df['phone_shops'] = phone_shops
df['phone_shops'].value_counts() 

100%|██████████| 532/532 [00:07<00:00, 68.78it/s]


0.0     149
1.0      57
3.0      54
2.0      49
4.0      46
6.0      39
5.0      35
7.0      24
9.0      21
10.0     14
8.0      13
12.0      8
13.0      4
11.0      4
19.0      3
35.0      2
15.0      2
18.0      1
16.0      1
29.0      1
21.0      1
34.0      1
14.0      1
24.0      1
32.0      1
Name: phone_shops, dtype: int64

In [208]:
df.head()

,point_id,lon,lat,target,city,downtown_dist,rail_station,airport,hotels,phone_shops
0,ommNZCUV,37.590776,55.848630,-0.348157,Москва,6.862762,0.0,0.0,2.0,7.0
1,nMe2LHPb,37.784210,55.750271,1.294206,Москва,6.482877,1.0,0.0,0.0,8.0
2,ZgodVRqB,39.635721,47.213330,-1.039679,Ростов-на-Дону,3.595842,0.0,0.0,1.0,0.0
3,0t2jNYdz,37.704570,55.782020,-1.169339,Москва,4.026767,0.0,0.0,3.0,6.0
4,U27W4QJ7,37.643983,55.730188,-0.088837,Москва,1.738010,1.0,0.0,9.0,12.0


In [254]:
df.airport = df.airport.astype(int)
df.rail_station = df.rail_station.astype(int)

In [255]:
df_train = df.iloc[:train.shape[0]]

In [256]:
df_test = df.iloc[train.shape[0]:]

# Fit model

In [257]:
from catboost import CatBoostRegressor, Pool, cv

In [258]:
model = CatBoostRegressor(logging_level='Silent')

In [259]:
categorical_features_indices = ['city', 'rail_station', 'airport']

In [266]:
X_train, X_valid, y_train, y_valid = train_test_split(df_train.drop(['target', 'point_id'], axis=1),
                                                      df_train['target'], test_size=0.2,
                                                      shuffle=True, random_state=852)
train_pool = Pool(X_train, y_train, cat_features=categorical_features_indices)
validate_pool = Pool(X_valid, y_valid, cat_features=categorical_features_indices)

In [267]:
model.fit(train_pool, eval_set=validate_pool)

In [268]:
mean_absolute_error(y_valid, model.predict(validate_pool))

0.46437150900837487

# Make submission

In [59]:
submission = pd.DataFrame()
submission['point_id'] = test['point_id']
submission['target'] = model.predict(df_test.drop(['point_id', 'target'], axis=1))
submission.to_csv('data/my_submission_01.csv', index=False)

# Some maps

In [218]:
import folium
from folium.plugins import HeatMap

In [249]:
def map_points(df, feature_df, city, plot_feature=True, \
               lat_col='lat', lon_col='lon', zoom_start=11, \
                plot_points=False, pt_radius=15, \
                draw_heatmap=False, heat_map_weights_col=None, \
                heat_map_weights_normalize=True, heat_map_radius=15):

    ## center map in the middle of points center in
    df = df[df['city']==city]
    feature_df = feature_df[feature_df['city']==city]
    
    middle_lat = df[lat_col].median()
    middle_lon = df[lon_col].median()
    
    curr_map = folium.Map(location=[middle_lat, middle_lon],
                          zoom_start=zoom_start)

    # add points to map
    if plot_points:
        for _, row in df.iterrows():
            folium.CircleMarker([row[lat_col], row[lon_col]],
                                radius=5,
                                color="#000000",
                                weight=1,
                               ).add_to(curr_map)

    # add heatmap
    if draw_heatmap:
        if heat_map_weights_col is None:
            cols_to_pull = [lat_col, lon_col]
        else:
            # if we have to normalize
            if heat_map_weights_normalize:
                df[heat_map_weights_col] = \
                    df[heat_map_weights_col] / df[heat_map_weights_col].sum()

            cols_to_pull = [lat_col, lon_col, heat_map_weights_col]

        stations = df[cols_to_pull]
        curr_map.add_children(HeatMap(stations, radius=heat_map_radius))
        
    if plot_feature:
        for _, row in feature_df.iterrows():
            folium.CircleMarker([row[lat_col], row[lon_col]],
                                radius=3,
                                color="#ff0000",
                                weight=1,
                               ).add_to(curr_map)

    return curr_map

## Красным отмечены отели, чёрным - салоны связи, heatmap по таргету

In [250]:
map_points(df_train, hotels_osm, city='Москва', plot_points=True,
           draw_heatmap=True, heat_map_weights_col='target')

## То же самое с rail stations

In [251]:
map_points(df_train, railways_osm, city='Москва', plot_points=True,
           draw_heatmap=True, heat_map_weights_col='target')

## Другие салоны связи

In [253]:
map_points(df_train, phone_shops_osm, city='Москва', plot_points=True,
           draw_heatmap=True, heat_map_weights_col='target')